# ***Clustering sobre Noticias de "El Tiempo" (Colombia)***

## ***Objetivo***

Identificar, analizar y resumir los temas predominantes en las noticias de "El Tiempo", con un enfoque en detectar patrones emergentes. A lo largo del ejercicio, se buscará la identificación de temas en busca de insights estratégicos sobre el interés público y los temas de impacto en Colombia.



## ***Pasos Detallados***

### ***1. Obtención de URLs de Noticias***
- **Uso de Web Scraping**: Recolecta URLs no solo de la sección de noticias recientes, sino de múltiples secciones como política, economía, deportes, y cultura.


### ***2. Extracción de resumenes del Texto de la Noticia***

- **Procesamiento Previo de Texto**: Realiza limpieza del texto (remoción de ruido, normalización, etc.).
- **Análisis de Sentimiento Inicial**: Como paso previo al clustering, utiliza una herramienta de análisis de sentimiento para agregar contexto emocional a los textos, lo cual puede enriquecer la interpretación de los clusters.

### 3. ***Generación de Embeddings con [`Ollama`](https://ollama.com/) y [`langchain`](https://python.langchain.com/docs/)***
- **Exploración Comparativa de Embeddings**: Genera embeddings utilizando diferentes modelos de Ollama para comparar la representatividad y consistencia de cada modelo en la agrupación de temas.
.

### ***4. Aplicación de Métodos de Clustering***
- **Exploración de Diferentes Algoritmos**: Experimenta con técnicas de clustering jerárquico, K-Means, GaussMixture.

- **Selección del Número de Clusters Dinámicamente**: Usa métodos como la silueta y Davies-Bouldin para determinar el número óptimo de clusters y realiza pruebas cruzadas.

- **Interpretación Visual**: Genera gráficos de reducción de dimensionalidad con UMAP y t-SNE para visualizar los clusters y detectar posibles temas subyacentes.

### ***5. Análisis con un LLM Local para Caracterización y Resumen de Clusters***

- **Uso de LLM para Insight Profundo**: Utiliza Ollama en Colab para generar resúmenes detallados y descripciones de cada cluster, incluyendo:

  - Resúmenes temáticos generales.
  - Identificación de términos clave.
  - Resumen de sentimientos predominantes dentro de cada cluster.


### ***6. Discusión y Evaluación de Resultados***

- **Contraste de Resultados con Noticias Internacionales**: Veirificar la perticencia de los clusters formados y realizar limpieza post modelado.
- **Generación de Reporte Visual e Interactivo**: Crea un tablero visual en Colab o herramientas como Streamlit para presentar los resultados, que incluya insights automáticos generados por el LLM para cada cluster.
- **Discusión y Conclusiones Estratégicas**: Propón hipótesis sobre por qué ciertos temas ganan o pierden relevancia en el periodo de análisis.


In [1]:
import requests
import pandas as pd
from io import StringIO

## ***Obtener urls de noticias del Tiempo***

Una sesión HTTP se es creada mediante `requests.Session()`. Las sesiones permiten mantener ciertas configuraciones y cookies a través de múltiples solicitudes.

In [2]:
url_new = (
    """https://www.eltiempo.com/deportes/otros-deportes/david-alonso-imparable-en-moto3-gano-en-indonesia-y-podria-asegurar"""
    """-el-titulo-en-la-proxima-carrera-en-japon-3385519"""
)

# Crear una sesión HTTP para mantener cookies y configuraciones en múltiples solicitudes
session = requests.Session()
# Peticion HTTP
response = session.get(url_new)
# Respuesta peticion
print("Respuesta a la peticion url:", response.status_code)
print(f"{100 * '='}")
# Contenido de la pagina web
print(response.text)

Respuesta a la peticion url: 200
<!DOCTYPE html>
    <html lang="es">
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
                                    

                                            
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta name="viewport" content="width=device-width, initial-scale=1">
<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
<title>David Alonso sigue imparable en Moto3: ganó en Indonesia y podría asegurar el título en la próxima carrera, en Japón</title>
<link href="https://www.eltiempo.com/deportes/otros-deportes/david-alonso-imparable-en-moto3-gano-en-indonesia-y-podria-asegurar-el-titulo-en-la-proxima-carrera-en-japon-3385519" rel="canonical">
<link rel="alternate" href="https://www.eltiempo.com/deportes/otros-deportes/david-alonso-imparable-en-moto3-gano-en-indonesia-y-podria-asegurar-el-titulo-en-la-proxima-carrera-en-japon-3385519" hreflang="es-co"/>
<meta name

### ***¿Qué es un Sitemap XML?***
Un sitemap es un archivo en formato XML que contiene una lista de URLs de un sitio web. Su propósito principal es ayudar a los motores de búsqueda a encontrar, rastrear e indexar todas las páginas importantes del sitio de manera más eficiente. Esto es especialmente útil para sitios grandes o aquellos que actualizan contenido con frecuencia.

La URL https://www.eltiempo.com/sitemap-articles-current.xml apunta al sitemap de El Tiempo, un periódico digital. En este archivo, podemos encontrar las URLs de artículos recientes, que ayudan a que los motores de búsqueda como Google puedan descubrir e indexar rápidamente el contenido nuevo o actualizado.

In [3]:
url = 'https://www.eltiempo.com/sitemap-articles-current.xml'
session = requests.Session()
response = session.get(url)
if response.raise_for_status:
    print(f"OK, with the page: {url}. Status Code: {response.status_code}\n")
    print(f"Text from Page:")
    print(response.text[: 1000])
else:
    print(f"Problem in the page: {url}")

OK, with the page: https://www.eltiempo.com/sitemap-articles-current.xml. Status Code: 200

Text from Page:
<?xml version="1.0" encoding="UTF-8"?>
<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9" xmlns:video="http://www.google.com/schemas/sitemap-video/1.1" xmlns:news="http://www.google.com/schemas/sitemap-news/0.9">
 <url>
  <loc>https://www.eltiempo.com/mundo/eeuu-y-canada/como-eliminar-las-manchas-de-cloro-en-la-ropa-con-sencillo-truco-3399643</loc>
  <lastmod>2024-11-14T12:00:19-05:00</lastmod>
  <news:news>
   <news:publication>
    <news:name>ElTiempo.com</news:name>
    <news:language>es</news:language>
   </news:publication>
   <news:title>Cómo eliminar las manchas de cloro en la ropa con sencillo truco</news:title>
   <news:publication_date>2024-11-14T12:00:19-05:00</news:publication_date>
   <news:keywords>cómo, eliminar, las, manchas, de</news:keywords>
  </news:news>
 </url>
 <url>
  <loc>https://www.eltiempo.com/colombia/barranquilla/encuentran-nuevo-tipo-de-bag

El siguiente código extrae URLs de un sitemap XML y las convierte en un DataFrame de pandas para facilitar su análisis.

- Espacios de nombres XML: Define los espacios de nombres para identificar etiquetas en el XML:

- ns es el espacio principal del sitemap.

- news y video son espacios adicionales para etiquetas de noticias y videos.
- Lectura del XML:
    - pd.read_xml() lee el XML, convirtiendo el contenido en un DataFrame.
    - StringIO(response.text) convierte el contenido de response.text en un objeto similar a un archivo, que pd.read_xml() necesita para procesar el XML.
    - xpath=".//ns:url" selecciona las etiquetas <url> en el XML.
    - namespaces=namespaces asegura que los espacios de nombres definidos sean reconocidos.

- El DataFrame df_urls_news_el_tiempo contiene las URLs y sus datos asociados (URL, fecha de modificación, prioridad, etc.), que se muestran como una tabla en el notebook.

In [4]:
namespaces = {
    "ns": "http://www.sitemaps.org/schemas/sitemap/0.9",
    "news": "http://www.google.com/schemas/sitemap-news/0.9",
    "video": "http://www.google.com/schemas/sitemap-video/1.1"
}

# Hacemos lectura del texto XML
df_urls_news_el_tiempo = pd.read_xml(StringIO(response.text), xpath=".//ns:url", namespaces=namespaces)
df_urls_news_el_tiempo

,loc,lastmod,news
0,https://www.eltiempo.com/mundo/eeuu-y-canada/c...,2024-11-14T12:00:19-05:00,\n
1,https://www.eltiempo.com/colombia/barranquilla...,2024-11-14T12:00:00-05:00,\n
2,https://www.eltiempo.com/deportes/futbol-inter...,2024-11-14T11:56:18-05:00,\n
3,https://www.eltiempo.com/cultura/gente/cuales-...,2024-11-14T11:54:02-05:00,\n
4,https://www.eltiempo.com/economia/sectores/la-...,2024-11-14T11:53:55-05:00,\n
...,...,...,...
3667,https://www.eltiempo.com/salud/cuanto-azucar-d...,2024-11-01T00:00:00-05:00,\n
3668,https://www.eltiempo.com/cultura/gente/video-l...,2024-11-01T00:00:00-05:00,\n
3669,https://www.eltiempo.com/deportes/futbol-colom...,2024-11-01T00:00:00-05:00,\n
3670,https://www.eltiempo.com/cultura/gente/cual-es...,2024-11-01T00:00:00-05:00,\n


In [5]:
# Hacemos lectura del texto XML
df_title_date_keywords_news_el_tiempo = pd.read_xml(StringIO(response.text), xpath=".//news:news", namespaces=namespaces)
df_title_date_keywords_news_el_tiempo

,publication,title,publication_date,keywords
0,\n,Cómo eliminar las manchas de cloro en la ropa ...,2024-11-14T12:00:19-05:00,"cómo, eliminar, las, manchas, de"
1,\n,"Carlos Ardila, el investigador que ha descubie...",2024-11-14T12:00:00-05:00,"Barranquilla, Sierra, Nevada, descubrimiento, ..."
2,\n,Futbolista Mauro Icardi recibió una denuncia p...,2024-11-14T11:56:18-05:00,"futbolista, mauro icardi, denuncia, wanda nara"
3,\n,¿Cuáles son las verduras que se deben comer cr...,2024-11-14T11:54:02-05:00,"cuáles, son, las, verduras, qué"
4,\n,La producción industrial de Colombia cayó en 4...,2024-11-14T11:53:55-05:00,"la, producción, industrial, de, colombia"
...,...,...,...,...
3667,\n,¿Cuánto azúcar diario puede consumir un niño p...,2024-11-01T00:00:00-05:00,"cuántos, dulces, puede, comer, un"
3668,\n,Video | 'Los Vengadores' se reunieron para mos...,2024-11-01T00:00:00-05:00,"video, los, vengadores, se, reúnen"
3669,\n,Video | Atlético Nacional dio un paso gigante ...,2024-11-01T00:00:00-05:00,"Nacional, Medellín, Copa BetPlay"
3670,\n,¿Cuál es la edad en la que los hombres y las m...,2024-11-01T00:00:00-05:00,"cuál, es, la, edad, en"


In [6]:
df_urls_news_el_tiempo = (
    df_urls_news_el_tiempo[["loc"]]
    .merge(
        df_title_date_keywords_news_el_tiempo.drop(columns=["publication"]),
        left_index=True,
        right_index=True        
    )
    .rename(columns={"loc": "url_page"})
)

df_urls_news_el_tiempo

,url_page,title,publication_date,keywords
0,https://www.eltiempo.com/mundo/eeuu-y-canada/c...,Cómo eliminar las manchas de cloro en la ropa ...,2024-11-14T12:00:19-05:00,"cómo, eliminar, las, manchas, de"
1,https://www.eltiempo.com/colombia/barranquilla...,"Carlos Ardila, el investigador que ha descubie...",2024-11-14T12:00:00-05:00,"Barranquilla, Sierra, Nevada, descubrimiento, ..."
2,https://www.eltiempo.com/deportes/futbol-inter...,Futbolista Mauro Icardi recibió una denuncia p...,2024-11-14T11:56:18-05:00,"futbolista, mauro icardi, denuncia, wanda nara"
3,https://www.eltiempo.com/cultura/gente/cuales-...,¿Cuáles son las verduras que se deben comer cr...,2024-11-14T11:54:02-05:00,"cuáles, son, las, verduras, qué"
4,https://www.eltiempo.com/economia/sectores/la-...,La producción industrial de Colombia cayó en 4...,2024-11-14T11:53:55-05:00,"la, producción, industrial, de, colombia"
...,...,...,...,...
3667,https://www.eltiempo.com/salud/cuanto-azucar-d...,¿Cuánto azúcar diario puede consumir un niño p...,2024-11-01T00:00:00-05:00,"cuántos, dulces, puede, comer, un"
3668,https://www.eltiempo.com/cultura/gente/video-l...,Video | 'Los Vengadores' se reunieron para mos...,2024-11-01T00:00:00-05:00,"video, los, vengadores, se, reúnen"
3669,https://www.eltiempo.com/deportes/futbol-colom...,Video | Atlético Nacional dio un paso gigante ...,2024-11-01T00:00:00-05:00,"Nacional, Medellín, Copa BetPlay"
3670,https://www.eltiempo.com/cultura/gente/cual-es...,¿Cuál es la edad en la que los hombres y las m...,2024-11-01T00:00:00-05:00,"cuál, es, la, edad, en"


## ***Obtener el texto de las noticias***

In [7]:
from bs4 import BeautifulSoup

In [8]:
# Ejemplo de url a visitar:
df_urls_news_el_tiempo.iloc[0].loc["url_page"]

'https://www.eltiempo.com/mundo/eeuu-y-canada/como-eliminar-las-manchas-de-cloro-en-la-ropa-con-sencillo-truco-3399643'

In [9]:
%%time
# Primeras 20 urls de noticias.
list_urls = df_urls_news_el_tiempo["url_page"].head(5).to_list()

for url in list_urls:
    print("Trabajando en la URL:", url)
    request = requests.get(url)
    soup = BeautifulSoup(request.content, "html.parser")
    
    # Verificar si el request es exitosa
    if request.raise_for_status:
        author_tag = soup.find("a", class_="c-detail__author__name").get_text()
        content = " ".join([tag.get_text() for tag in soup.find_all("div", class_="paragraph")])     
        print(f"Autor: {author_tag}")
        print(f"Contenido noticia: {content}")
        print()

Trabajando en la URL: https://www.eltiempo.com/mundo/eeuu-y-canada/como-eliminar-las-manchas-de-cloro-en-la-ropa-con-sencillo-truco-3399643


Autor: Donato Del Blanco
Contenido noticia: Las manchas de cloro en la ropa pueden ser una pesadilla para cualquier persona que valore su guardarropa. Aunque es un aliado potente en la limpieza, su uso indebido puede arruinar prendas que, de otro modo, estarían en perfecto estado. Sin embargo, no todo está perdido. Si sufrió un accidente con este producto, existen técnicas eficaces que pueden ayudar a minimizar o incluso eliminar las manchas, según el tipo de prenda y el grado de daño. Según la página oficial de Ariel, es importante entender que las manchas de cloro no son manchas tradicionales, como las de vino o grasa. Este actúa eliminando el color de la tela, en lugar de dejar una mancha visible con pigmentos. Cuando entra en contacto con un tejido, destruye los tintes que dan color a la tela, lo que resulta en una marca blanca o descolorida. La rapidez con que esto sucede depende de la concentración del producto y el tiempo de exposición. Si bien esto puede ser frustrante, también

Autor: Leonardo Herrera Delgans
Contenido noticia: Bajo el reflejo del río Sevilla, en la majestuosa Sierra Nevada de Santa Marta, se encuentra un pez que ahora es símbolo de décadas de pasión y esfuerzo.  (Además : Las autoridades en Barranquilla reportan baja en hurtos : la inversión en equipos y tecnología, una de las claves) Esta es la historia de Trichomycterus caribensis, una especie única y reciente en el mundo de la ictiología, y de Carlos Ardila Rodríguez, el hombre que dedicó su vida a desvelar los secretos de los peces de agua dulce en Colombia.  Con este hallazgo, Ardila, profesor de la Universidad Metropolitana de Barranquilla,  ha alcanzado la cifra de cien especies descubiertas, un logro que lo confirma como quizás el más prolífico de los científicos colombianos en su campo. Desde niño, cuando apenas contaba con cinco años, Carlos Ardila sintió el llamado del agua. En su natal Floridablanca (Santander), los arroyos de su infancia, como la Judía Chiquita y el río Frío, fu

Autor: Elim Johana Alonso Dorado
Contenido noticia: Luego de que el futbolista Mauro Icardi decidiera regresar a Argentina tras su lesión, una rotura de ligamentos y menisco de la rodilla, e instalarse en el edificio que comparte con Wanda Nara, en Chateau Libertador, el futbolista recibió una denuncia por parte de su expareja por violencia de género. (Además: Wissam ben Yedder, una joyita: el futbolista condenado a dos años de cárcel en suspenso por agresión sexual). Según indicaron medios argentinos, la conductora del programa 'Bake Off' pidió que el futbolista desalojara el apartamento que comparten, dado que Wanda encontró que él había tomado posesión de este, luego de que regresara de sus vacaciones en Brasil con su actual pareja. También, informaron que cuando la presentadora llegó al apartamento, Icardi no solo impidió que ingresara a este, sino que se inició una acalorada discusión. Por esa razón, Nara se retiró a otra propiedad que tiene en Nordelta y pidió una orden de desalo

Autor: Tania Alejandra Hernández Torres
Contenido noticia: Las rutinas alimenticias ricas en frutas y verduras siempre son recomendadas por los especialistas en nutrición. Estos alimentos de origen natural ofrecen al cuerpo vitaminas, nutrientes y minerales que los diferentes sistemas no son capaces de producir por sí solos. La Clínica Mayo menciona que este grupo alimenticio ofrece al cuerpo muchas vitaminas, minerales, antioxidantes, fibra, agua, fotoquímicos, entre otros. Sin embargo, existen ciertas formas en las que deben ser consumidas para no perder los beneficios que aportan al cuerpo. Le puede interesar: Habla el hijo de Karina, exmodelo de Stock Models, que ahora es habitante de calle: ‘Ojalá se pueda recuperar’ El portal ‘Cuerpo Mente’ menciona que las verduras contienen muchos componentes valiosos y beneficiosos para la salud, pero explica que al cocinarlas y someterlas a altas temperaturas muchos de estos nutrientes se pierden. Este sitio web especializado en salud y biene

Autor: Laura Lesmes Díaz
Contenido noticia: Durante septiembre de 2024, la producción Industrial de Colombia presentó una variación negativa de 4,5 por ciento, frente a las cifras del mismo periodo del año pasado. Así lo reveló el Departamento Nacional de Estadística, Dane en el Índice de Producción Industrial. Además en la Encuesta Mensual Manufacturera se revela una reducción en la producción real del 4,2 por ciento. En el noveno mes del 2024 frente al 2023, los sectores presentaron variaciones negativas, la mayor fue el de explotación de minas y canteras con 9,2 por ciento; seguido por la industria manufacturera  que se redujo en 4,2 por ciento;  la captación, tratamiento y distribución de agua  tuvo una variación negativa de 1,2 por ciento y el suministro de electricidad y gas se redujo también  en 0,6 por ciento. En septiembre de 2024 frente al año pasado, de las 26 actividades industriales 22 presentaron variaciones negativas restando 5,0 puntos porcentuales a la variación anual 

## ***Paralelizar la tarea de obtencion del contenido de las noticias***

In [10]:
session = requests.Session()

def get_content_news_from_url(url: str) -> dict:
    """
    Obtiene el contenido de una noticia desde una URL, extrayendo el autor y el cuerpo de la noticia.

    :param url: URL de la página de la noticia.
    :return: Diccionario con la URL, autor y contenido de la noticia.
    :raises ValueError: Si hay un problema con el status_code de la solicitud.
    """
    request = session.get(url)
    
    if request.raise_for_status:
        soup = BeautifulSoup(request.content, "html.parser")
        author_tag = soup.find("a", class_="c-detail__author__name").get_text()
        content = " ".join([tag.get_text() for tag in soup.find_all("div", class_="paragraph")])
        
        return {
            "url_page": url,
            "autor": author_tag,
            "news_content": content            
        }
    else:
        ValueError(
            f"Problemas con el status_code: {request.status_code}"
        )
        

get_content_news_from_url(df_urls_news_el_tiempo.iloc[0].loc["url_page"]) 

{'url_page': 'https://www.eltiempo.com/mundo/eeuu-y-canada/como-eliminar-las-manchas-de-cloro-en-la-ropa-con-sencillo-truco-3399643',
 'autor': 'Donato Del Blanco',
 'news_content': 'Las manchas de cloro en la ropa pueden ser una pesadilla para cualquier persona que valore su guardarropa. Aunque es un aliado potente en la limpieza, su uso indebido puede arruinar prendas que, de otro modo, estarían en perfecto estado. Sin embargo, no todo está perdido. Si sufrió un accidente con este producto, existen técnicas eficaces que pueden ayudar a minimizar o incluso eliminar las manchas, según el tipo de prenda y el grado de daño. Según la página oficial de Ariel, es importante entender que las manchas de cloro no son manchas tradicionales, como las de vino o grasa. Este actúa eliminando el color de la tela, en lugar de dejar una mancha visible con pigmentos. Cuando entra en contacto con un tejido, destruye los tintes que dan color a la tela, lo que resulta en una marca blanca o descolorida. La

In [11]:
import os
from typing import Dict
from concurrent.futures import ThreadPoolExecutor, as_completed

num_cores = os.cpu_count()
print(f"Número de cores disponibles: {num_cores}")

# Es recomendado dejar un core libre
# num_cores = num_cores - 1


def parallel_process_urls(urls: list[str]) -> list[Dict]:
    """
    Procesa múltiples URLs en paralelo utilizando ThreadPoolExecutor,
    dejando 2 cores libres.
    
    :param urls: Lista de URLs a procesar.
    :return: Lista de diccionarios con la información extraída de cada URL.
    """
    results = []
    
    with ThreadPoolExecutor(max_workers=num_cores) as executor:
        futures = [executor.submit(get_content_news_from_url, url) for url in urls]
        
        for future in as_completed(futures):
            try:
                results.append(future.result())
            except Exception as e:
                print(f"Error procesando la URL: {e}")
    
    return results

# Lista de URLs a procesar. Descomente la siguiente linea de codigo 
# para obtener todas las urls.
# Procesamos todas las URLs en paralelo
# urls_to_process = df_urls_news_el_tiempo["url_page"].tolist()

# urls de ejemplo, solo una muestra de 20 urls
urls_to_process = df_urls_news_el_tiempo["url_page"].sample(20).tolist()
# Procesamos todas las URLs en paralelo
processed_results = parallel_process_urls(urls_to_process)
# Resultados
print(processed_results)
pd.DataFrame(processed_results)

Número de cores disponibles: 16


[{'url_page': 'https://www.eltiempo.com/mundo/eeuu-y-canada/esta-habilidad-puede-pagar-jugoso-sueldo-al-ano-en-ee-uu-y-no-requiere-un-titulo-3395922', 'autor': 'Shareni Pastrana', 'news_content': 'Según los expertos y las estadísticas recientes de bolsas de trabajo en línea, tener la habilidad de utilizar redes sociales para implementar técnicas de marketing digital puede hacer que usted comience a ganar sueldos de hasta seis cifras en Estados Unidos, sin necesidad de salir de casa. Además para realizar este tipo de labores no se requiere de un título. CNBC Make It asegura que el auge del marketing en redes sociales se disparó gracias al crecimiento del comercio electrónico y los cambios en el comportamiento del consumidor después de la pandemia.\xa0 Asimismo, el medio citado afirma que saber utilizar TikTok, Instagram y otras redes sociales puede convertirse en su mayor oportunidad para crecer de manera rápida en el ámbito laboral, ya que “las empresas están ansiosas por contratar per

,url_page,autor,news_content
0,https://www.eltiempo.com/mundo/eeuu-y-canada/e...,Shareni Pastrana,Según los expertos y las estadísticas reciente...
1,https://www.eltiempo.com/justicia/delitos/el-c...,Carlos López,"En las antiguas instalaciones del DAS, en Bogo..."
2,https://www.eltiempo.com/mundo/eeuu-y-canada/i...,Agencia EFE,Las autoridades de diversos estados de EE. UU....
3,https://www.eltiempo.com/mundo/europa/volodimi...,afp,"El presidente ucraniano, Volodimir Zelenski, a..."
4,https://www.eltiempo.com/bogota/racionamiento-...,María Camila Salas Valencia,La Empresa de Acueducto y Alcantarillado de Bo...
5,https://www.eltiempo.com/vida/medio-ambiente/u...,Alejandra López Plazas,Luego de que en respuesta a las emergencias oc...
6,https://www.eltiempo.com/colombia/santander/pi...,María Camila Salas Valencia,"Este viernes 8 de noviembre de 2024, la medida..."
7,https://www.eltiempo.com/deportes/futbol-inter...,Pablo Romero,Una noticia 'bomba' explotó este viernes en Eu...
8,https://www.eltiempo.com/colombia/santander/im...,Sebastián David García Castro,Las lluvias en el departamento de Santander qu...
9,https://www.eltiempo.com/justicia/jep-colombia...,Sara Valentina Quevedo Delgado,En el trasfondo de los ruidos que en los últim...


In [12]:
df_news_el_tiempo = df_urls_news_el_tiempo.merge(
    pd.DataFrame(processed_results),
    on=["url_page"]    
)

display(df_news_el_tiempo)
# Descomente las siguientes lineas de codigo se desea guarda los resultados
# en un archivo parquet
# df_news_el_tiempo.to_parquet("data/df_noticias_el_tiempo.parquet")
# pd.read_parquet("data/df_noticias_el_tiempo.parquet")

,url_page,title,publication_date,keywords,autor,news_content
0,https://www.eltiempo.com/justicia/cortes/corte...,Corte Constitucional tumbó la norma que permit...,2024-11-13T11:38:29-05:00,"corte, constitucional, tumbó, la, norma",Jesús Antonio Blanquicet,La Corte Constitucional acaba de tumbar la exp...
1,https://www.eltiempo.com/bogota/racionamiento-...,Racionamiento de agua en Bogotá: estos son los...,2024-11-12T16:11:29-05:00,"Bogotá, Cortes, Agua, Acueducto, Racionamiento",María Camila Salas Valencia,La Empresa de Acueducto y Alcantarillado de Bo...
2,https://www.eltiempo.com/politica/congreso/fra...,"Francia Márquez, a debate de control político ...",2024-11-12T00:00:00-05:00,"Francia Márquez, Control Político",Mateo García Agudelo,"La vicepresidenta y ministra de Igualdad, Fran..."
3,https://www.eltiempo.com/salud/la-tuberculosis...,La tuberculosis habría superado al covid-19 y ...,2024-11-12T00:00:00-05:00,"la, tuberculosis, habría, superado, al",Agencia EFE,Tras ser superada durante tres años consecutiv...
4,https://www.eltiempo.com/economia/empresas/iva...,‘IVA de 19 % a los carros híbridos podría fren...,2024-11-11T11:32:26-05:00,"Hyundai, Hyundai colombia, carros Hyundai, ven...",Noelia Cigüenza Riaño,La marca surcoreana Hyundai ha ido ganando poc...
5,https://www.eltiempo.com/vida/medio-ambiente/u...,UNGRD destinará 2.500 millones de pesos para a...,2024-11-10T18:46:51-05:00,"ungrd, destinará, 2, 500, millones",Alejandra López Plazas,Luego de que en respuesta a las emergencias oc...
6,https://www.eltiempo.com/colombia/santander/im...,Impresionantes imágenes de emergencia en San V...,2024-11-10T13:30:00-05:00,"San Vicente de Chucurí, Santander, lluvias, av...",Sebastián David García Castro,Las lluvias en el departamento de Santander qu...
7,https://www.eltiempo.com/mundo/eeuu-y-canada/i...,Investigan campaña de mensajes anónimos racist...,2024-11-09T02:30:00-05:00,"investigan, campaña, de, mensajes, anónimos",Agencia EFE,Las autoridades de diversos estados de EE. UU....
8,https://www.eltiempo.com/deportes/futbol-inter...,Selección Brasil iría en serio por Pep Guardio...,2024-11-08T13:18:19-05:00,"Pep Guardiola, Brasil",Pablo Romero,Una noticia 'bomba' explotó este viernes en Eu...
9,https://www.eltiempo.com/mundo/europa/primer-m...,Primer ministro de Irlanda llama a elecciones ...,2024-11-08T08:55:17-05:00,"Irlanda, Simon Harris, elecciones, elecciones ...",efe,"El primer ministro irlandés, Simon Harris, con..."


## ***Usar un LLM local***

In [13]:
# Instalar Ollama (disponible para Windows, Linux y MacOS)
# Aca se debe poner la guia de instalacion en Colab (pendiente)
# 1. Instalemos Ollama en Colab --> !curl -fsSL https://ollama.com/install.sh | sh
# 2. Verificamos su instalacion --> !ollama --version
# 3. Instalamos una terminal en Colab para ejecutar Ollama
#       !pip install colab-xterm
#       %load_ext colabxterm
#       %xterm
# 4. Servir Ollama: run en terminal ---> ollama serve
# 5. Descargar modelo: !ollama pull qwen2.5:7b
# 6. Probar modelo: 
#       %xterm
#       !ollama run qwen2.5:7b

In [14]:
# !curl -fsSL https://ollama.com/install.sh | sh

In [15]:
# !ollama --version

In [16]:
## Installemos una terminal en Colab para ejecutar Ollama
#!pip install colab-xterm
#%load_ext colabxterm
#%xterm
##run in terminal ---> ollama serve

In [17]:
# !ollama pull qwen2.5:7b

In [18]:
# %xterm
# ollama run qwen2.5:7b

In [19]:
#!pip install langchain-ollama==0.2.0

In [20]:
# Generar un chat con un llm_ollama
from langchain_ollama import ChatOllama

llm_ollama = ChatOllama(
    model="qwen2.5:7b",
    temperature=0.1,
    num_predict=1024,
    keep_alive=0
)

llm_response = llm_ollama.invoke("¿Cuál es la segunda letra del alfabeto griego?")
print(llm_response)

content='La segunda letra del alfabeto griego es "Beta" (Β ηβ), que se escribe así en mayúscula y así en minúscula.' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2024-11-14T21:12:56.292183948Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 8489319905, 'load_duration': 5971223782, 'prompt_eval_count': 43, 'prompt_eval_duration': 1290000000, 'eval_count': 38, 'eval_duration': 789000000} id='run-683201fb-7d8b-4554-a3ff-57a54da0b282-0' usage_metadata={'input_tokens': 43, 'output_tokens': 38, 'total_tokens': 81}


In [21]:
%%time
llm_ollama.invoke(
    """Dame un lista de 20 personajes influyentes en la actualidad"""
    """mundial. No olvides incluir diferentes angulos de la vida. """
    """Solo debes darme la lista sin decir nada mas. Por ejemplo si """
    """decides incluir a Pepito Perez (suponiendo que es influyente """
    """actualmente) respuesta debe ser: Pepito Perez"""
    )

CPU times: user 39 ms, sys: 751 μs, total: 39.8 ms
Wall time: 6.04 s


AIMessage(content='Bill Gates\nElon Musk\nAngela Merkel\nJoe Biden\nXi Jinping\nBeyoncé\nCristiano Ronaldo\nGisele Bündchen\nPope Francis\nOprah Winfrey\nJack Ma\nMalala Yousafzai\nIndira Gandhi\nNeymar Jr.\nTaylor Swift\nJK Rowling\nMark Zuckerberg\nSalma Hayek\nBernie Sanders\nAmal Clooney', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2024-11-14T21:13:02.345900758Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6041233497, 'load_duration': 2705189100, 'prompt_eval_count': 102, 'prompt_eval_duration': 889000000, 'eval_count': 86, 'eval_duration': 1955000000}, id='run-ec6e3c79-e9d3-4767-8ca5-733804770991-0', usage_metadata={'input_tokens': 102, 'output_tokens': 86, 'total_tokens': 188})

In [22]:
# PromptTemplate y respuesta como string
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

prompt = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate.from_template(
            """Eres una util AI bot que crea excelentes biografias de """
            """personajes influyentes de la actualidad mundial y detalla su """
            """vida en diferentes aspectos. Responde usando solo JSON con """
            """llave el nombre del personaje y valor su biografia, """
            """Por ejemplo: dict(Pepito Perez="Pepito Peres ... (biografia))"""),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

print(prompt)

chain = prompt | llm_ollama | StrOutputParser()
chain.invoke(
    {
        "user_input": "Barack Obama"
    }
)

input_variables=['user_input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres una util AI bot que crea excelentes biografias de personajes influyentes de la actualidad mundial y detalla su vida en diferentes aspectos. Responde usando solo JSON con llave el nombre del personaje y valor su biografia, Por ejemplo: dict(Pepito Perez="Pepito Peres ... (biografia))'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})]


'{\n  "Barack Obama": "Barack Obama es un político estadounidense que sirvió como el 44º presidente de los Estados Unidos desde 2009 hasta 2017. Nació en Honolulu, Hawái, el 4 de agosto de 1961, y fue educado en varias universidades, incluyendo Occidental College, la Universidad de Columbia y la Universidad Harvard, donde obtuvo un doctorado en Derecho y Ciencias Políticas. En su carrera política anterior a la presidencia, Obama trabajó como profesor de derecho y se involucró activamente en asuntos legales y políticos. Fue elegido senador por Illinois en 2004 y luego candidato presidencial demócrata en 2008, ganando la nominación y derrotando al republicano John McCain para convertirse en el primer presidente afroamericano de los Estados Unidos. Durante su mandato, Obama implementó reformas significativas en áreas como la salud, la educación y las energías renovables, y enfrentó desafíos globales como la crisis financiera mundial y la guerra contra el terrorismo. Después de dejar la pr

In [23]:
# Usamos JSON-format para parsear la respuesta del LLM
from langchain_core.output_parsers import JsonOutputParser

json_llm = ChatOllama(
    model="qwen2.5:7b",
    temperature=0.1,
    num_predict=-1,
    format="json",
    keep_alive=0
)

prompt = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate.from_template(
            """Eres una util AI bot que crea excelentes biografias de """
            """personajes influyentes de la actualidad mundial y detalla su """
            """vida ampliamente en diferentes aspectos. No debes escatimar """
            """en la longitud del texto de resumen. Responde usando solo JSON """
            """format con llave el nombre del personaje y valor su biografia"""
            """Por ejemplo: dict(Pepito Perez="Pepito Perez ... (biografia)","""
            """ Benito Camelas="Benito Camelas ... (biografia)"). Debes """
            """seguir al pie de la letra el formato"""
            ),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

chain = prompt | json_llm | JsonOutputParser()
llm_answer = chain.invoke({"user_input": "Dame la biografia de 2 personajes"})
print(llm_answer, type(llm_answer))

{'Bill Gates': "Bill Gates, nacido el 28 de octubre de 1955 en Seattle, Washington, es un empresario, filántropo y escritor estadounidense. Es co-fundador de Microsoft Corporation junto con su amigo y colega Paul Allen, donde ocupó el cargo de presidente hasta 1981 y luego como director ejecutivo hasta 2000. Gates se retiró oficialmente de Microsoft en 2008 para dedicarse a la fundación Bill & Melinda Gates, una organización sin fines de lucro que se centra en mejorar la salud global y reducir la desigualdad económica. Además de su trabajo filantrópico, Gates es autor del libro 'The Road to Prosperity', donde expone sus ideas sobre cómo los países pueden superar la pobreza. Su fortuna personal, estimada en alrededor de 132 mil millones de dólares según Forbes, lo convierte en uno de los hombres más ricos del mundo.", 'Salma Hayek': "Salma Hayek, nacida el 8 de junio de 1966 en Ciudad de México, es una reconocida actriz y productora mexicana. Comenzó su carrera en la televisión mexicana

### ***Embeddings***

In [24]:
# !ollama pull mxbai-embed-large:latest

In [25]:
%%time
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
)

question = ["¿Cuál es la segunda letra del alfabeto griego?"]
embed_question = ollama_embeddings.embed_documents(question)
print("Embeddings:")
pd.DataFrame(embed_question, index=question)

Embeddings:
CPU times: user 151 ms, sys: 1.82 ms, total: 153 ms
Wall time: 2.88 s


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
¿Cuál es la segunda letra del alfabeto griego?,-0.048147,0.015149,0.041317,0.001429,-0.037248,-0.011467,0.014369,0.00321,0.02435,-0.0077,...,0.045875,-0.015134,0.012814,-0.031493,0.009488,0.033627,-0.001471,0.034937,-0.060316,-0.0377


In [26]:
# Ejemplo: (Relevancia - Distancia)
from sklearn.metrics.pairwise import cosine_similarity
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
)

texts = [
    "Alpha es la primera letra del alfabeto griego",
    "A es la primera letra del alfabeto latino",
    "Beta es la segunda letra del alfabeto griego",
    "B es la segunda letra del alfabeto latino"
]
query = ["¿Cuál es la segunda letra del alfabeto griego?"]
embeds = ollama_embeddings.embed_documents(texts)
embed_query = ollama_embeddings.embed_documents(query)
print(f"Long Ollama Embeddings: {len(embed_query[0])}")
print(f"Cosine Similarities: \n {cosine_similarity(embeds, embed_query)}")

pd.DataFrame(
    cosine_similarity(embeds, embed_query),
    index=texts,
    columns=query
).sort_values(by=query, ascending=False)

Long Ollama Embeddings: 1024
Cosine Similarities: 
 [[0.76785541]
 [0.77705817]
 [0.89837255]
 [0.85569265]]


,¿Cuál es la segunda letra del alfabeto griego?
Beta es la segunda letra del alfabeto griego,0.898373
B es la segunda letra del alfabeto latino,0.855693
A es la primera letra del alfabeto latino,0.777058
Alpha es la primera letra del alfabeto griego,0.767855


In [27]:
def get_summary_news(news: str, model_name: str = "qwen2.5:7b") -> str:

    llm_ollama = ChatOllama(
        model=model_name,
        temperature=0.1,
        num_predict=4096,
        num_ctx=16000,
        keep_alive=0
    )

    prompt_summarization = ChatPromptTemplate(
        [
            SystemMessagePromptTemplate.from_template(
                """Por favor, crea un resumen natural, claro y preciso de esta noticia. Asegúrate de incluir los 
                siguientes puntos de manera fluida::"""
                """1. Evento principal o tema central: Describe en una o dos frases el evento clave o el tema más """
                """importante de la noticia."""
                """2. Actores principales y sus reacciones: Menciona las partes involucradas y sus posturas o 
                reacciones significativas (gobiernos, empresas, expertos, etc.)."""
                """3. Consecuencias o implicaciones: Señala las implicaciones principales o cualquier posible """
                """consecuencia a corto o largo plazo derivada de la noticia."""
                """4. Contexto adicional relevante: Incluye cualquier información de contexto que sea relevante para """
                """comprender mejor el evento o las decisiones tomadas."""
                """El resumen debe ser natural y no exceder las cinco frases. Evita juicios o interpretaciones, 
                    enfocándote en capturar la esencia de la noticia sin perder detalles importantes."""
            ),
            HumanMessagePromptTemplate.from_template("El texto de la noticias es:\n{news}")
        ]
    )

    llm_chain = prompt_summarization | llm_ollama | StrOutputParser()

    return llm_chain.invoke({"news": news})

In [28]:
df_test = df_news_el_tiempo.sample(2)
df_test = df_test[["url_page", "news_content"]]
df_test["summary"] = df_test["news_content"].apply(get_summary_news)
df_test

,url_page,news_content,summary
0,https://www.eltiempo.com/justicia/cortes/corte...,La Corte Constitucional acaba de tumbar la exp...,La Corte Constitucional ha eliminado la expres...
13,https://www.eltiempo.com/cultura/gente/poner-u...,"Muchas personas huyen a la cocina, y no solo p...",El tema central de la noticia es un truco para...


In [29]:
for _, row in df_test.iterrows():
    print(f"URL: {row['url_page']}")
    print(f"Noticia: {row['news_content']}")
    print(f"Resumen: {row['summary']}\n")

URL: https://www.eltiempo.com/justicia/cortes/corte-constitucional-tumbo-la-norma-que-permitia-las-mutilaciones-en-animales-por-razones-esteticas-3399260
Noticia: La Corte Constitucional acaba de tumbar la expresión “estética” contenida en el literal c del artículo 6 de la Ley 84 de 1989, “por la cual se adopta el Estatuto Nacional para la protección de los animales”, toda vez que la palabra “desconoce” el mandato constitucional de protección animal. (Lea: El expediente de Hernán Giraldo, el depredador sexual de las Auc que el presidente Petro nombró gestor de paz) La demanda señalaba que la disposición parcialmente demandada “permitía el maltrato animal” y consideraba que, por ese motivo, desconoce los artículos 7º, 79 y 95 de la Constitución Política, que establecen la obligación de protección a la fauna, la prohibición de maltrato a los animales y el deber de proteger los recursos naturales y velar por la conservación del ambiente sano. Tal postura fue respaldada por el  Ministerio 

In [30]:
# Embedding de las noticias.
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
)

embed_news = ollama_embeddings.embed_documents(df_test["summary"].to_list())
print("Embeddings:")
pd.DataFrame(embed_news, index=df_test["url_page"])

Embeddings:


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
url_page,,,,,,,,,,,,,,,,,,,,,
https://www.eltiempo.com/justicia/cortes/corte-constitucional-tumbo-la-norma-que-permitia-las-mutilaciones-en-animales-por-razones-esteticas-3399260,-0.011709,-0.015197,0.012366,-0.002624,-0.012364,-0.028625,-0.025987,0.005309,-0.022448,0.044481,...,0.038573,-0.009798,-0.012597,0.014054,0.007458,0.016252,-0.042067,0.024714,-0.011502,-0.013165
https://www.eltiempo.com/cultura/gente/poner-una-cuchara-de-madera-sobre-la-olla-evita-que-se-riegue-la-leche-al-hervirla-la-ciencia-explica-por-que-funciona-el-truco-3396779,-0.042442,0.028711,0.009765,0.036304,0.001532,-0.023031,-0.026659,0.033387,0.053019,0.037357,...,0.032277,0.004599,-0.015712,-0.007293,0.038114,0.023659,-0.016401,0.019705,-0.005332,0.015621
